In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

In [3]:
spark = SparkSession.builder.appName('Master').getOrCreate()

In [4]:
df = spark.read.csv('master.csv', inferSchema = True, header = True)

In [5]:
jumlahData = df.count()
print('Jumlah Data = ', jumlahData)
df.printSchema()

Jumlah Data =  27820
root
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- country-year: string (nullable = true)
 |-- HDI for year: double (nullable = true)
 |--  gdp_for_year ($) : string (nullable = true)
 |-- gdp_per_capita ($): integer (nullable = true)
 |-- generation: string (nullable = true)



# Menampilkan isi dataset master.csv

In [6]:
df.show(30)

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|        null|     2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4

# Menampilkan Data dengan Populasi lebih dari 200.000

In [10]:
DataFrame = df.select(df['Country'],
                      df['Sex'],
                      df['Year'],
                      df['Population'])
DataFrame.filter(DataFrame['Population'] > 200000).show(100) #Menampilkan Populasi yang lebih dari 200000

+-------+------+----+----------+
|Country|   Sex|Year|Population|
+-------+------+----+----------+
|Albania|  male|1987|    312900|
|Albania|  male|1987|    308000|
|Albania|female|1987|    289700|
|Albania|  male|1987|    274300|
|Albania|female|1987|    278800|
|Albania|female|1987|    257200|
|Albania|female|1987|    311000|
|Albania|  male|1987|    338200|
|Albania|  male|1988|    319200|
|Albania|  male|1988|    314100|
|Albania|female|1988|    295600|
|Albania|female|1988|    262400|
|Albania|  male|1988|    279900|
|Albania|female|1988|    284500|
|Albania|female|1988|    317200|
|Albania|  male|1988|    345000|
|Albania|  male|1989|    283600|
|Albania|  male|1989|    318400|
|Albania|  male|1989|    323500|
|Albania|female|1989|    288600|
|Albania|female|1989|    299900|
|Albania|female|1989|    266300|
|Albania|female|1989|    321900|
|Albania|  male|1989|    349700|
|Albania|  male|1992|    343800|
|Albania|  male|1992|    263700|
|Albania|  male|1992|    245500|
|Albania|f

# Menghapus kolom country-year dan HDI for year karena tidak dibutuhkan.

In [7]:
df.drop('country-year', 'HDI for year').show()

+-------+----+------+-----------+-----------+----------+-----------------+------------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19|     2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4.59|     2,156,624,900|               796|G.I. Generation|
|Albania|1987|  male|25-34 years|          9|    274300|             3.28|     2,156,624,900|    

In [8]:
#Membuat sql table temporary dari dataframe yang dibuat
df.createOrReplaceTempView("suicide")

Sqlsuicide = spark.sql("SELECT * FROM suicide")
Sqlsuicide.show()

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|        null|     2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4

# Menampilkan data bunuh diri berdasarkan tahun dari 2015 sampai 2016

In [9]:
df2 = df.filter( (col('year') >= lit('2015')) & (col('year') <= lit('2016')) )
df2.groupBy("country","year").sum("suicides_no").orderBy('year').show(50)

+--------------------+----+----------------+
|             country|year|sum(suicides_no)|
+--------------------+----+----------------+
|         Puerto Rico|2015|             226|
|          Luxembourg|2015|              64|
|          Seychelles|2015|               7|
|              Sweden|2015|            1182|
|            Thailand|2015|            4205|
|             Georgia|2015|             192|
|             Ukraine|2015|            7574|
|             Belgium|2015|            1867|
|              Panama|2015|             110|
|              Mexico|2015|            6234|
|             Uruguay|2015|             630|
|             Romania|2015|            2228|
|               Chile|2015|            1838|
|              Norway|2015|             590|
|           Australia|2015|            3027|
|             Estonia|2015|             195|
|               Qatar|2015|              66|
|              Israel|2015|             342|
|             Hungary|2015|            1868|
|         

# Menampilkan kasus bunuh diri terbanyak berdasar tahun

In [11]:
df.groupBy("year").sum("suicides_no").orderBy('sum(suicides_no)', ascending=False).show()

+----+----------------+
|year|sum(suicides_no)|
+----+----------------+
|1999|          256119|
|2002|          256095|
|2003|          256079|
|2000|          255832|
|2001|          250652|
|1998|          249591|
|1996|          246725|
|1995|          243544|
|2009|          243487|
|2004|          240861|
|1997|          240745|
|2010|          238702|
|2011|          236484|
|2008|          235447|
|2005|          234375|
|2007|          233408|
|2006|          233361|
|1994|          232063|
|2012|          230160|
|2013|          223199|
+----+----------------+
only showing top 20 rows



# Menampilkan negara dengan rata-rata suicide tertinggi (Unggulan 2)

In [98]:
df.groupBy("country").avg("suicides_no").orderBy('avg(suicides_no)', ascending=False).show()

+------------------+------------------+
|           country|  avg(suicides_no)|
+------------------+------------------+
|Russian Federation|3733.7716049382716|
|     United States|2779.6048387096776|
|             Japan|2169.0913978494623|
|           Ukraine| 952.2321428571429|
|           Germany| 933.5320512820513|
|            France| 914.2416666666667|
| Republic of Korea| 703.5752688172043|
|            Brazil| 609.1747311827957|
|            Poland| 482.9791666666667|
|         Sri Lanka|421.52272727272725|
|    United Kingdom|  367.755376344086|
|             Italy|             355.0|
|          Thailand|331.26646706586826|
|        Kazakhstan|325.46794871794873|
|            Canada| 309.0833333333333|
|            Mexico|298.76075268817203|
|             Spain|269.36021505376345|
|           Hungary|238.35806451612902|
|           Belarus|237.66666666666666|
|         Argentina| 221.0188172043011|
+------------------+------------------+
only showing top 20 rows



# Menampilkan negara dengan kasus suicide terbanyak

In [71]:
df.groupBy("country").sum("suicides_no").orderBy('sum(suicides_no)', ascending=False).show()

+------------------+----------------+
|           country|sum(suicides_no)|
+------------------+----------------+
|Russian Federation|         1209742|
|     United States|         1034013|
|             Japan|          806902|
|            France|          329127|
|           Ukraine|          319950|
|           Germany|          291262|
| Republic of Korea|          261730|
|            Brazil|          226613|
|            Poland|          139098|
|    United Kingdom|          136805|
|             Italy|          132060|
|            Mexico|          111139|
|          Thailand|          110643|
|            Canada|          107561|
|        Kazakhstan|          101546|
|             Spain|          100202|
|         Argentina|           82219|
|           Hungary|           73891|
|           Romania|           72777|
|         Australia|           70111|
+------------------+----------------+
only showing top 20 rows



# Menampilkan umur dan gender yang beresiko suicide (Unggulan 2)

In [72]:
df.groupBy("sex", "age").sum("suicides_no").orderBy('sum(suicides_no)', ascending=False).show()

+------+-----------+----------------+
|   sex|        age|sum(suicides_no)|
+------+-----------+----------------+
|  male|35-54 years|         1945908|
|  male|55-74 years|         1228407|
|  male|25-34 years|          915089|
|  male|15-24 years|          633105|
|female|35-54 years|          506233|
|  male|  75+ years|          431134|
|female|55-74 years|          430036|
|female|  75+ years|          221984|
|female|25-34 years|          208823|
|female|15-24 years|          175437|
|  male| 5-14 years|           35267|
|female| 5-14 years|           16997|
+------+-----------+----------------+



# Menampilkan Jumlah bunuh diri berdasarkan generasi dan gender

In [8]:
df.groupBy("generation","sex").sum("suicides_no").orderBy('sum(suicides_no)', ascending=False).show(50)

+---------------+------+----------------+
|     generation|   sex|sum(suicides_no)|
+---------------+------+----------------+
|        Boomers|  male|         1823530|
|         Silent|  male|         1309455|
|   Generation X|  male|         1222965|
|     Millenials|  male|          489839|
|         Silent|female|          472289|
|        Boomers|female|          460968|
|G.I. Generation|  male|          333356|
|   Generation X|female|          309839|
|G.I. Generation|female|          176653|
|     Millenials|female|          133620|
|   Generation Z|  male|            9765|
|   Generation Z|female|            6141|
+---------------+------+----------------+



# Generasi pemberi kontribusi suicide terbanyak

In [85]:
df.groupBy("generation").sum("suicides_no").orderBy('sum(suicides_no)', ascending=False).show()

+---------------+----------------+
|     generation|sum(suicides_no)|
+---------------+----------------+
|        Boomers|         2284498|
|         Silent|         1781744|
|   Generation X|         1532804|
|     Millenials|          623459|
|G.I. Generation|          510009|
|   Generation Z|           15906|
+---------------+----------------+



# Menampilkan pengaruh gdp perkapita negara terhadap tingkat suicide

In [87]:
economicFactor = df.groupBy('country','year').sum('gdp_per_capita ($)', 'suicides_no' ).orderBy('country').show()

+-------+----+-----------------------+----------------+
|country|year|sum(gdp_per_capita ($))|sum(suicides_no)|
+-------+----+-----------------------+----------------+
|Albania|1992|                   3012|              47|
|Albania|2001|                  17412|             119|
|Albania|1989|                   9996|              68|
|Albania|1994|                   8364|              50|
|Albania|1993|                   5244|              73|
|Albania|2000|                  15588|              54|
|Albania|2005|                  35172|               0|
|Albania|2006|                  38820|               0|
|Albania|1996|                  13524|              89|
|Albania|1999|                  13524|             139|
|Albania|1997|                   9516|             170|
|Albania|1988|                   9228|              63|
|Albania|2003|                  24252|             124|
|Albania|2008|                  56064|             160|
|Albania|2009|                  52644|          

# Menampilkan perbandingan tingkat suicide pertahun dari tiap negara

In [89]:
df.groupBy('country','year').sum('gdp_per_capita ($)', 'suicides_no' ).orderBy('year', ascending=False).show()

+--------------+----+-----------------------+----------------+
|       country|year|sum(gdp_per_capita ($))|sum(suicides_no)|
+--------------+----+-----------------------+----------------+
|        Cyprus|2016|                 250980|              36|
|        Sweden|2016|                 555940|            1130|
|       Hungary|2016|                 134480|            1761|
|      Thailand|2016|                  67130|            4117|
|Czech Republic|2016|                 195050|            1318|
|       Grenada|2016|                 108380|               0|
|       Romania|2016|                 100200|            1953|
|       Croatia|2016|                 129050|             683|
|         Qatar|2016|                 624840|              68|
|       Armenia|2016|                  37880|              67|
|      Mongolia|2016|                  41450|             423|
|       Iceland|2016|                 647080|              40|
|     Mauritius|2016|                 105700|          

# Menampilkan perbandingan tingkat suicide dengan gdp per kapita setiap negara pada tiap tahunnya

In [81]:
df.groupBy('country','year').sum('gdp_per_capita ($)', 'suicides_no' ).orderBy('country','year').show()

+-------+----+-----------------------+----------------+
|country|year|sum(gdp_per_capita ($))|sum(suicides_no)|
+-------+----+-----------------------+----------------+
|Albania|1987|                   9552|              73|
|Albania|1988|                   9228|              63|
|Albania|1989|                   9996|              68|
|Albania|1992|                   3012|              47|
|Albania|1993|                   5244|              73|
|Albania|1994|                   8364|              50|
|Albania|1995|                  10020|              88|
|Albania|1996|                  13524|              89|
|Albania|1997|                   9516|             170|
|Albania|1998|                  10788|             154|
|Albania|1999|                  13524|             139|
|Albania|2000|                  15588|              54|
|Albania|2001|                  17412|             119|
|Albania|2002|                  18876|             133|
|Albania|2003|                  24252|          